## LCEL

The most basic and common use case is chaining a prompt template and a model together. To see how this works, let's create a chain that takes a topic and generates a joke:

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a poem about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "the moon"})

"In the velvet cloak of night,  \nThe moon hangs high, a silver light,  \nA lantern in the vast expanse,  \nWhispering secrets in a dance.  \n\nHer glow is soft, a tender balm,  \nA soothing spell, a tranquil calm,  \nShe bathes the world in gentle beams,  \nAwakening the night with dreams.  \n\nThe stars, they twinkle, bow in grace,  \nWhile shadows play in her embrace,  \nThe trees stand tall, their whispers low,  \nAs moonlit rivers weave and flow.  \n\nShe pulls the tides, a silent guide,  \nA guardian of the ocean's stride,  \nIn every phase, from crescent new,  \nTo full and bright, her magic's true.  \n\nOh, moon of silver, calm and bright,  \nYou cradle hopes in your soft light,  \nA muse for lovers, a friend to night,  \nIn your embrace, the world feels right.  \n\nSo here we stand, beneath your glow,  \nIn awe of all the love you show,  \nForever in your gentle sway,  \nThe moon shall light the night away.  "

Notice this line of the code, where we piece together these different components into a single chain using LCEL:

`chain = prompt | model | output_parser`

The | symbol is similar to a unix pipe operator, which chains together the different components, feeding the output from one component as input into the next component.

In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model, then the model output is passed to the output parser. Let's take a look at each component individually to really understand what's going on.

## Prompt

In [4]:
prompt_value = prompt.invoke({"topic": "the moon"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a poem about the moon', additional_kwargs={}, response_metadata={})])

In [5]:
prompt_value.to_messages()

[HumanMessage(content='tell me a poem about the moon', additional_kwargs={}, response_metadata={})]

In [6]:
prompt_value.to_string()

'Human: tell me a poem about the moon'

## Model
`PromptValue` is passed sequentially to `model`. This will output a `BaseMessage`. 

In [7]:
message = model.invoke(prompt_value)
message

AIMessage(content='In the velvet cloak of night,  \nThe moon ascends, a silver light,  \nA watchful eye in skies so deep,  \nWhere dreams are born and secrets keep.  \n\nShe bathes the world in gentle glow,  \nKissing the rivers, soft and slow,  \nWhispers of stories, old and wise,  \nReflect in her calm, ethereal rise.  \n\nWith craters etched like tales long told,  \nOf cosmic dances, brave and bold,  \nShe pulls the tides with tender grace,  \nA silent guardian of time and space.  \n\nIn her embrace, the shadows play,  \nWhile starlit whispers drift away,  \nA tapestry of night unfurls,  \nAs she weaves magic through the swirls.  \n\nOh, luminous orb of distant skies,  \nYou hold our hopes, our starlit sighs,  \nWith every phase, a chance to dream,  \nIn your soft glow, we find our gleam.  \n\nSo here beneath your watchful gaze,  \nWe pause, we ponder, we explore the maze,  \nFor in your light, we find our tune,  \nForever enchanted by the moon.  ', additional_kwargs={'refusal': Non

In [8]:
#A model as an LLM will output a string
from langchain_openai import OpenAI

llm = OpenAI()
llm.invoke(prompt_value)

"\n\nIn the dark of night, she shines so bright\nA silvery orb, a mystical sight\nThe moon, a celestial wonder above\nGuiding sailors, and inspiring love\n\nShe waxes and wanes, a constant dance\nA symbol of change, of second chance\nShe pulls the tides, and rules the sea\nBut her true power, is in her mystery\n\nA lonely traveler, in the vast expanse\nHer cratered surface, a poetic trance\nShe watches over us, with a tranquil grace\nA calming presence, in this chaotic space\n\nThe moon, a muse for poets and dreamers\nHer gentle glow, a source of gleamers\nA beacon of hope, in the darkest of night\nA reminder of beauty, in every sight\n\nSo let us gaze, at her glowing face\nAnd lose ourselves, in her quiet embrace\nFor the moon, in all her celestial glory\nIs a timeless symbol, of nature's story."

## Output parser
And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. The specific StrOutputParser simply converts any input into a string.

In [9]:
output_parser.invoke(message)

'In the velvet cloak of night,  \nThe moon ascends, a silver light,  \nA watchful eye in skies so deep,  \nWhere dreams are born and secrets keep.  \n\nShe bathes the world in gentle glow,  \nKissing the rivers, soft and slow,  \nWhispers of stories, old and wise,  \nReflect in her calm, ethereal rise.  \n\nWith craters etched like tales long told,  \nOf cosmic dances, brave and bold,  \nShe pulls the tides with tender grace,  \nA silent guardian of time and space.  \n\nIn her embrace, the shadows play,  \nWhile starlit whispers drift away,  \nA tapestry of night unfurls,  \nAs she weaves magic through the swirls.  \n\nOh, luminous orb of distant skies,  \nYou hold our hopes, our starlit sighs,  \nWith every phase, a chance to dream,  \nIn your soft glow, we find our gleam.  \n\nSo here beneath your watchful gaze,  \nWe pause, we ponder, we explore the maze,  \nFor in your light, we find our tune,  \nForever enchanted by the moon.  '

## The Pipeline
To follow the steps along:

- We pass in user input on the desired topic as `{"topic": "ice cream"}`
- The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a `ChatMessage` object.
- Finally, the `output_parser` component takes in a `ChatMessage`, and transforms this into a Python string, which is returned from the invoke method.

<img src="../images/chain_image.png">

In [10]:
input = {"topic": "the moon"}

print(prompt.invoke(input))


print((prompt | model).invoke(input))


messages=[HumanMessage(content='tell me a poem about the moon', additional_kwargs={}, response_metadata={})]
content='In the velvet sky, a lantern glows,  \nWhispers of silver where the night wind blows.  \nA guardian of dreams, in tranquil embrace,  \nThe moon weaves her magic, a soft, tender lace.  \n\nShe dances on waters with a shimmering grace,  \nPainting the shadows, a celestial trace.  \nHer beams like a lullaby, gentle and sweet,  \nGuiding the lost on their winding retreat.  \n\nIn the hush of the night, when the world holds its breath,  \nShe cradles the secrets, the stories of death.  \nFrom lovers’ soft sighs to the cries of the trees,  \nThe moon listens closely, a witness to pleas.  \n\nOh, how she changes, a fickle old friend,  \nFrom crescent to full, on her cycles depend.  \nA beacon of solace in darkness profound,  \nIn her quiet presence, true peace can be found.  \n\nSo here’s to the moon, in her luminous flight,  \nA symbol of hope in the depths of the night.  \nM

## Using RAG

In [11]:

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

/home/gauntletpaul/class-5-chaining-with-lcel/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'

In [12]:
chain = setup_and_retrieval | prompt | model | output_parser

To explain this, we first can see that the prompt template above takes in context and question as values to be substituted in the prompt. Before building the prompt template, we want to retrieve relevant documents to the search and include them as part of the context.

As a preliminary step, we’ve setup the retriever using an in memory store, which can retrieve documents based on a query. This is a runnable component that can be chained together with other components, but you can also try to run it separately:

In [13]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [14]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [15]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

The flow being:

- The first steps create a RunnableParallel object with two entries. The first entry, context will include the document results fetched by the retriever. The second entry, question will contain the user’s original question. To pass on the question, we use RunnablePassthrough to copy this entry.
- Feed the dictionary from the step above to the prompt component. It then takes the user input which is question as well as the retrieved document which is context to construct a prompt and output a PromptValue.
- The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation. The generated output from the model is a ChatMessage object.
- Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

<img src="../images/chain_parallel.png">

## Advantages

### Invoke

In [16]:
from typing import List

import openai


prompt_template = "Tell me a short joke about {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("ice cream")

'Why did the ice cream truck break down? \n\nBecause it couldn’t find its “sundae” driver!'

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o-mini")
chain = (
    {"topic": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream truck break down?  \n\nBecause it had a rocky road!'

### Stream  

In [18]:
from typing import Iterator


def stream_chat_model(messages: List[dict]) -> Iterator[str]:
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content

def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt.format(topic=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])


for chunk in stream_chain("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream truck break down? 

Because it had too many scoop-ache!

In [19]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream truck break down? 

Because it had a rocky road!

### Batch

In [20]:
from concurrent.futures import ThreadPoolExecutor


def batch_chain(topics: list) -> list:
    with ThreadPoolExecutor(max_workers=5) as executor:
        return list(executor.map(invoke_chain, topics))

batch_chain(["ice cream", "spaghetti", "dumplings"])

['What did the ice cream cone say to the refrigerator? \n\n“I’ve got to chill out!”',
 'Why did the spaghetti break up with the meatball? \n\nBecause it couldn’t handle the sauce of the relationship!',
 "Why did the dumpling go to therapy?  \n\nBecause it couldn't stop feeling so stuffed!"]

In [21]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

['What did the ice cream cone say to the scoop? \n\n"I scream, you scream, we all scream for ice cream!" 🍦',
 'Why did the spaghetti break up with the meatball? \n\nBecause it couldn’t handle the pressure!',
 'Why did the dumpling break up with the noodle?  \n\nBecause it found someone who really knew how to wrap it up!']

### Async

In [22]:
async_client = openai.AsyncOpenAI()

async def acall_chat_model(messages: List[dict]) -> str:
    response = await async_client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
    )
    return response.choices[0].message.content

async def ainvoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return await acall_chat_model(messages)


await ainvoke_chain("ice cream")

'Why did the ice cream cone become a chef?  \n\nBecause it wanted to make "scoop"-tacular dishes!'

In [23]:
await chain.ainvoke("ice cream")

'Why did the ice cream cone break up with the sundae? \n\nBecause it found someone cooler! 🍦'